ecommerce\src\ecommerce\forms.py

In [ ]:
from django import forms
from django.contrib.auth import get_user_model

User = get_user_model()

class UserRegisterForm(forms.ModelForm):
    password = forms.CharField(
        widget=forms.PasswordInput,
        label="Contraseña"
    )
    password2 = forms.CharField(
        widget=forms.PasswordInput,
        label="Confirmar contraseña"
    )

    class Meta:
        model = User
        fields = ["username", "email"]

    def clean(self):
        cleaned_data = super().clean()
        p1 = cleaned_data.get("password")
        p2 = cleaned_data.get("password2")

        if p1 and p2 and p1 != p2:
            self.add_error("password2", "Las contraseñas no coinciden")

        return cleaned_data

    def save(self, commit=True):
        user = super().save(commit=False)
        user.set_password(self.cleaned_data["password"])
        if commit:
            user.save()
        return user


ecommerce\src\ecommerce\urls.py

In [ ]:
from django.urls import path
from .views import (
    VentasHomeView,
    AddToCartView,
    RemoveFromCartView,
    ClearCartView,
    CheckoutView,
    VentasAjaxView,
    register
    
)

app_name = "ecommerce"

urlpatterns = [
    path("", VentasHomeView.as_view(), name="ventas-home"),
    path("cart/add/<int:product_id>/", AddToCartView.as_view(), name="add-to-cart"),
    path("cart/remove/<int:product_id>/", RemoveFromCartView.as_view(), name="remove-from-cart"),
    path("cart/clear/", ClearCartView.as_view(), name="clear-cart"),
    path("checkout/", CheckoutView.as_view(), name="checkout"),
    path("sales/data/", VentasAjaxView.as_view(), name="sales-data"),
    path("register/", register, name="register"),

]


ecommerce\src\ecommerce\views.py

In [ ]:
from django.shortcuts import render, redirect
from django.contrib import messages

from .forms import UserRegisterForm


def register(request):
    initial_data = {
        "username": "usuario_demo",
        "email": "correo@demo.com",
    }

    form = UserRegisterForm(
        request.POST or None,
        initial=initial_data
    )

    if form.is_valid():
        form.save()
        messages.success(request, "Usuario creado correctamente")
        return redirect("ecommerce:home")  
    return render(request, "ecommerce/register.html", {
        "form": form
    })

ecommerce\src\templates\ecommerce\register.html

In [ ]:
{% extends "base.html" %}

{% block content %}
<h1>Registro de usuario</h1>

<form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Registrarse</button>
</form>
{% endblock %}
